In [1]:
from bs4 import BeautifulSoup

def extract_race_result(file_path):
    # HTMLファイルを読み込む
    with open(file_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")

    # レース結果のテーブルを取得
    result_table = soup.find("table", class_="RaceTable01")

    if not result_table:
        print("レース結果のテーブルが見つかりませんでした。")
        return

    # テーブルの行を取得
    rows = result_table.find_all("tr")[1:]  # ヘッダー行をスキップ

    return rows

def extract_horse_names(rows):
    # 着順と馬名を抽出
    for row in rows:
        columns = row.find_all("td")
        if len(columns) >= 4:  # 必要なカラム数を確認
            rank = columns[0].text.strip()  # 着順
            horse_name = columns[3].text.strip()  # 馬名
            print(f"着順: {rank}, 馬名: {horse_name}")

In [42]:
# レース結果分析
import os
target_race_name = "test"  # 保存先のディレクトリ名
year = 2025
filepath = f'race_results{os.sep}{target_race_name}{os.sep}race_result_{year}.html'

rows = extract_race_result(filepath)
if rows:
    extract_horse_names(rows)

着順: 1, 馬名: シュトラウス
着順: 2, 馬名: マイネルモーント
着順: 3, 馬名: ルージュリナージュ
着順: 4, 馬名: グランドカリナン
着順: 5, 馬名: グリューネグリーン
着順: 6, 馬名: シンリョクカ
着順: 7, 馬名: ウンブライル
着順: 8, 馬名: トーセンリョウ
着順: 9, 馬名: ホウオウノーサイド
着順: 10, 馬名: グランスラムアスク
着順: 11, 馬名: トゥデイイズザデイ
着順: 12, 馬名: レガトゥス
着順: 13, 馬名: アドマイヤハレー


In [12]:
import requests
import os


def get_race_result(base_url, target_year, place, time, day, race):
    place_dict = {
        "01": "sapporo",
        "02": "hakodate",
        "03": "fukushima",
        "04": "niigata",
        "05": "tokyo",
        "06": "nakayama",
        "07": "chukyo",
        "08": "kyoto",
        "09": "hanshin",
        "10": "kokura",
    }
    url = f"{base_url}{target_year}{place}{time}{day}{race}"
    print(f"URL: {url}")
    # User-Agentを指定
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:121.0) Gecko/20100101 Firefox/121.0"
    }

    # target_race_nameのディレクトリが存在しない場合作成
    target_dir = f"race_results{os.sep}{target_year}{os.sep}{place_dict[place]}{os.sep}{time}{os.sep}{day}"
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    file_name = f"{target_dir}{os.sep}{race}.html"

    # ファイルがすでに存在している場合はスキップ
    if os.path.exists(file_name):
        print(f"{file_name} はすでに存在します。スキップします。")
        return 0, True

    # HTTP GETリクエストを送信し、HTMLを取得
    response = requests.get(url, headers=headers)

    # ステータスコードが200（成功）でない場合はエラーを表示
    if response.status_code != 200:
        print("ページの取得に失敗しました。")
        print(f"ステータスコード: {response.status_code}")
    else:
        # レスポンス内容を.htmlファイルとして保存
        # レスポンスのエンコーディングを明示的に設定
        response.encoding = (
            response.apparent_encoding
        )  # サーバーが指定するエンコーディングを推測

        # HTMLをBeautiful Soupでパース（エンコーディングを指定）
        soup = BeautifulSoup(response.content, "html.parser", from_encoding="utf-8")

        # レース結果のテーブルを取得
        result_table = soup.find("table", class_="RaceTable01")

        # レース結果のテーブルが存在する場合のみレスポンスの内容をファイルに書き込む
        if result_table:
            with open(file_name, "w", encoding="utf-8") as file:
                file.write(response.text)
            print(f"{file_name} に保存しました。")
            return 0, False
        else:
            print(f"{file_name}にはレース結果のテーブルが見つかりませんでした。")
            return 1, False


In [31]:
# 年単位ですべてのレース結果を取得する場合
def get_race_results_per_year(target_year):
    import time as t
    base_url = "https://race.netkeiba.com/race/result.html?race_id="

    # 最初の2桁は開催場所
    # （01：札幌、02：函館、03：福島、04：新潟、05:東京、06：中山、07：中京、08：京都、09：阪神、10：小倉）
    # 次の2桁は開催数
    # 次の2桁は開催日
    # 最後の2桁はレース番号
    # 例：06010111←中山開催1回目1日目11レース

    places = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"]
    times = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"]
    days = ["01", "02", "03", "04", "05", "06", "07", "08"]
    races = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

    # すべての組み合わせを取得
    result_not_found_tmp = 0
    day_end = None
    save_times = 0
    for place in places[:]:
        if result_not_found_tmp > 0:
            result_not_found_tmp = 0
        for time in times:
            if result_not_found_tmp > 0:
                result_not_found_tmp = 0
                # 開催1日目の結果が存在しない→今年度の開催はまだここまで、次の開催地に遷移
                if day_end == "01":
                    print(f"開催1日目の結果が存在しないため、次の開催地に遷移します。")
                    day_end = None
                    break
            for i, day in enumerate(days):
                if result_not_found_tmp > 0:
                    # レース結果が存在しない開催日を記録
                    day_end = days[i-1]
                    break
                for race in races:
                    result_not_found_tmp, skip_flg = get_race_result(base_url, target_year, place, time, day, race)
                    save_times += 1
                    # ファイルがすでに存在する場合は待機せずに次のレース結果を取得
                    if skip_flg:
                        continue
                    if save_times % 1 == 0:
                        save_times = 0
                        t.sleep(1)  # 1秒待機
                    # レース結果が見つからない→これ以上のレース結果の検索は中止、次の開催日or開催地に遷移
                    if result_not_found_tmp > 0:
                        print(f"レース結果が見つからなかったため、次に遷移します。")
                        break




In [32]:
target_year_start = 2020
target_year_end = 2025

for target_year in range(target_year_start, target_year_end + 1):
    print(f"=== {target_year}年のレース結果を取得 ===")
    get_race_results_per_year(target_year)

=== 2020年のレース結果を取得 ===
URL: https://race.netkeiba.com/race/result.html?race_id=202001010101
race_results/2020/sapporo/01/01/01.html に保存しました。
URL: https://race.netkeiba.com/race/result.html?race_id=202001010102
race_results/2020/sapporo/01/01/02.html に保存しました。
URL: https://race.netkeiba.com/race/result.html?race_id=202001010103
race_results/2020/sapporo/01/01/03.html に保存しました。
URL: https://race.netkeiba.com/race/result.html?race_id=202001010104
race_results/2020/sapporo/01/01/04.html に保存しました。
URL: https://race.netkeiba.com/race/result.html?race_id=202001010105
race_results/2020/sapporo/01/01/05.html に保存しました。
URL: https://race.netkeiba.com/race/result.html?race_id=202001010106
race_results/2020/sapporo/01/01/06.html に保存しました。
URL: https://race.netkeiba.com/race/result.html?race_id=202001010107
race_results/2020/sapporo/01/01/07.html に保存しました。
URL: https://race.netkeiba.com/race/result.html?race_id=202001010108
race_results/2020/sapporo/01/01/08.html に保存しました。
URL: https://race.netkeiba.com/ra

ConnectionError: HTTPSConnectionPool(host='race.netkeiba.com', port=443): Max retries exceeded with url: /race/result.html?race_id=202004010611 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x77330d473ee0>: Failed to resolve 'race.netkeiba.com' ([Errno -3] Temporary failure in name resolution)"))

In [23]:
filepath = "/home/naoki/Documents/keiba_analysis/race_results/nakayama_kinpai/race_result_2020.html"
rows = extract_race_result(filepath)
if rows:
    for row in rows:
        columns = row.find_all('td')
        print(len(columns))

レース結果のテーブルが見つかりませんでした。
